<a href="https://colab.research.google.com/github/Ryder-24/Project-for-Fall-2023-for-Tools-for-Analytics/blob/main/20news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.6334679500000675
GPU (s):
0.10668389100010245
GPU speedup over CPU: 34x


In [ ]:
!pip install bertopic
!pip install datasets
!pip install openai
!pip install octis

In [ ]:
all-mpnet-base-v2
all-MiniLM-L12-v2

In [ ]:
!git clone https://github.com/MaartenGr/BERTopic_evaluation.git
%cd BERTopic_evaluation
!git checkout 18f9aaf8872012a93b704f225080c32466d0160d
%ls evaluation


fatal: destination path 'BERTopic_evaluation' already exists and is not an empty directory.
/content/BERTopic_evaluation
HEAD is now at 18f9aaf Init commit
data.py  evaluation.py  __init__.py  __pycache__/  results.py


In [ ]:
%cd /content/BERTopic_evaluation
%ls
%cd evaluation
%ls


/content/BERTopic_evaluation
evaluation/  LICENSE  notebooks/  README.md  results/  setup.py


In [ ]:
from data import DataLoader
from evaluation import Trainer
from sentence_transformers import SentenceTransformer

In [ ]:
dataset, custom = "20NewsGroup", False
data_loader = DataLoader(dataset)
_, timestamps = data_loader.load_docs()
data = data_loader.load_octis(custom)
data = [" ".join(words) for words in data.get_corpus()]




Batches:   0%|          | 0/510 [00:00<?, ?it/s]

In [ ]:
# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(data, show_progress_bar=True)
for i in range(3):
    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        #"diversity": None,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"BERTopic_mpnet_20NewsGroup_{i+1}")

2024-04-19 03:06:48,842 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-19 03:07:23,173 - BERTopic - Dimensionality - Completed ✓
2024-04-19 03:07:23,176 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-19 03:07:23,897 - BERTopic - Cluster - Completed ✓
2024-04-19 03:07:23,898 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-19 03:07:24,824 - BERTopic - Representation - Completed ✓
2024-04-19 03:07:24,825 - BERTopic - Topic reduction - Reducing number of topics
2024-04-19 03:07:25,709 - BERTopic - Topic reduction - Reduced number of topics from 78 to 10


Results
npmi: 0.10546491684816564
diversity: 0.7666666666666667
 


2024-04-19 03:07:33,917 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-19 03:07:49,699 - BERTopic - Dimensionality - Completed ✓
2024-04-19 03:07:49,701 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-19 03:07:50,338 - BERTopic - Cluster - Completed ✓
2024-04-19 03:07:50,340 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-19 03:07:51,257 - BERTopic - Representation - Completed ✓
2024-04-19 03:07:51,259 - BERTopic - Topic reduction - Reducing number of topics
2024-04-19 03:07:52,122 - BERTopic - Topic reduction - Reduced number of topics from 84 to 20


Results
npmi: 0.11399228104741266
diversity: 0.7789473684210526
 


2024-04-19 03:08:03,684 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-19 03:08:18,820 - BERTopic - Dimensionality - Completed ✓
2024-04-19 03:08:18,822 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-19 03:08:19,480 - BERTopic - Cluster - Completed ✓
2024-04-19 03:08:19,481 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-19 03:08:20,391 - BERTopic - Representation - Completed ✓
2024-04-19 03:08:20,393 - BERTopic - Topic reduction - Reducing number of topics
2024-04-19 03:08:21,253 - BERTopic - Topic reduction - Reduced number of topics from 87 to 30


...

Results
npmi: 0.11635737626042962
diversity: 0.7591836734693878
 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2024-04-19 03:09:34,721 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


KeyboardInterrupt: 

In [ ]:
# Extract embeddings
model = SentenceTransformer("all-MiniLM-L12-v2")
embeddings = model.encode(data, show_progress_bar=True)
for i in range(3):
    params = {
        "embedding_model": "all-MiniLM-L12-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        #"diversity": None,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"BERTopic_MiniLM_20NewsGroup_{i+1}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/510 [00:00<?, ?it/s]

2024-04-19 03:17:47,803 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-19 03:18:02,039 - BERTopic - Dimensionality - Completed ✓
2024-04-19 03:18:02,041 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-19 03:18:02,704 - BERTopic - Cluster - Completed ✓
2024-04-19 03:18:02,706 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-19 03:18:03,605 - BERTopic - Representation - Completed ✓
2024-04-19 03:18:03,607 - BERTopic - Topic reduction - Reducing number of topics
2024-04-19 03:18:04,444 - BERTopic - Topic reduction - Reduced number of topics from 87 to 10


Results
npmi: 0.12011018114005388
diversity: 0.8777777777777778
 


2024-04-19 03:18:12,649 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-19 03:18:26,860 - BERTopic - Dimensionality - Completed ✓
2024-04-19 03:18:26,863 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-19 03:18:27,606 - BERTopic - Cluster - Completed ✓
2024-04-19 03:18:27,608 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-19 03:18:28,538 - BERTopic - Representation - Completed ✓
2024-04-19 03:18:28,539 - BERTopic - Topic reduction - Reducing number of topics
2024-04-19 03:18:29,368 - BERTopic - Topic reduction - Reduced number of topics from 88 to 20


KeyboardInterrupt: 